In [22]:
!pip install langchain openai chromadb tiktoken

In [23]:
!wget https://github.com/chatgpt-kr/chatgpt-api-tutorial/raw/main/ch05/data.zip
!unzip data

--2023-09-18 08:30:24--  https://github.com/chatgpt-kr/chatgpt-api-tutorial/raw/main/ch05/data.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/chatgpt-kr/chatgpt-api-tutorial/main/ch05/data.zip [following]
--2023-09-18 08:30:25--  https://raw.githubusercontent.com/chatgpt-kr/chatgpt-api-tutorial/main/ch05/data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 835816 (816K) [application/zip]
Saving to: ‘data.zip.1’

data.zip.1          100%[===================>] 816.23K  --.-KB/s    in 0.009s  

2023-09-18 08:30:25 (87.5 MB/s) - ‘data.zip.1’ saved [835816/835816]

Archive:  data.zip
replace

In [38]:
import os
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader, DirectoryLoader
from collections import Counter

os.environ['OPENAI_API_KEY'] = ""

In [39]:
loader = DirectoryLoader('.', glob="*.txt", loader_cls=TextLoader)
documents = loader.load()
print('문서의 개수 :', len(documents))

문서의 개수 : 57


In [40]:
print("1번 문서 :", documents[1])
print("-" * 20)
print("21번 문서 :", documents[21])
print("-" * 20)

1번 문서 : page_content='정책 이름: 창조적 예비청년창업가(로컬 크리에이터)\n서울신용보증재단에서는 골목상권에서 지역 가치를 차별화된 콘텐츠로 구현하고 매력적인 도시 문화를 형성할 수 있는 로컬 크리에이터를 양성하기 위한 창업 지원 정책을 진행합니다. 이 프로그램은 5개월 동안 교육을 지원하며, 약 10개 팀 정도가 지원을 받을 수 있습니다. 신청자격은 19세에서 39세까지의 서울시 거주자이며, 로컬 브랜드 상권 2기 대상지에서 창업을 희망하는 예비 창업자여야 합니다. 학력, 전공, 취업 상태, 특화 분야에 대한 요건은 없습니다. 신청 방법, 신청 절차, 심사 및 발표 관련 사항, 제출 서류 등은 추후 공지를 통해 안내될 예정입니다. 이 프로그램은 서울신용보증재단에서 운영됩니다.' metadata={'source': '4.txt'}
--------------------
21번 문서 : page_content='정책 제목: 지역주도형 청년일자리\n정책 유형은 "일자리"이며, 이 정책은 서울시청 일자리정책과에서 주관하고 있습니다. 해당 정책은 청년에게 적합한 지역 일자리를 발굴하고 제공하여 지역사회 정착 기반을 마련하고, 청년의 일 경험 기회 제공과 향후 구직활동과의 연계를 지원하는 것을 목표로 합니다. 지원 내용은 4개 유형별로 상이하며, 각 유형은 지역혁신형(개편1유형), 지역포용형(개편3유형), 지역정착지원형(기존1유형), 지역포스트코로나대응형(기존4유형)으로 구분됩니다. 사업 운영기간은 2023년 1월 1일부터 2023년 12월 31일까지이며, 신청은 2023년 1월 1일부터 2023년 12월 31일까지 가능합니다. 지원규모는 총 219명으로, 시, 구, 투출기관을 통해 2,800명의 참여자와 민간기업 맞춤형으로 900명의 참여자를 지원합니다. 신청자격은 19세부터 39세까지이며, 학력, 전공, 취업상태, 특화분야에는 제한이 없습니다. 신청 방법은 사업 수행 구청/사업부서의 개별 안내에 따라 진행되며, 심사 및 발표 역시 해당 구청/사업부서에

In [41]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

print('분할된 텍스트 개수:', len(texts))

분할된 텍스트 개수: 64


In [42]:
print(texts[0])

page_content='정책제목: 신혼부부 임차보증금 지원\n정부에서는 주거 관련 정책을 통해 부담을 완화하여 더 나은 주거환경을 제공하고자 합니다. 현재 주거마련에 대한 부담으로 인해 혼인수가 감소하고 출산기피 현상이 발생하고 있습니다. 따라서 주거비 부담을 완화하여 이러한 문제를 해결하고, 좋은 주거환경을 제공하고자 합니다.\n\n이 정책은 서울시청 주택정책과에서 주관하며, 주거 마련에 대한 부담을 완화하기 위한 내용을 포함하고 있습니다. 지원 대상은 관내 임차보증금 7억 이내의 주택 또는 주거용 오피스텔에 대해 해당하는 서울시민이나 서울로 전입 예정인 자입니다. 대출한도는 임차보증금의 90% 이내 또는 2억원 중 작은 금액이며, 대출금의 최대 연 3.6% 이차보전 및 최장 10년까지 지원됩니다.\n\n주택조건과 대출형식은 한국주택금융공사 보증 및 협약은행(국민, 하나, 신한) 대출, 그리고 서울시 이차보전이 적용됩니다. 이 정책은 2023년 1월 1일부터 2023년 12월 31일까지 운영되며, 신청자격은 1세부터 100세까지의 연령을 가진 사람들이 해당합니다.\n\n대출을 받기 위한 추가 요건으로는 혼인신고일 기준으로 7년 이내의 신혼부부이거나 서울시 추천서 신청일로부터 6개월 이내에 결혼식 예정인 예비신혼부부여야 합니다. 또한 부부합산 연소득이 9천 7백만원 이하이고 본인 및 배우자가 무주택자여야 합니다. 대출을 받을 주택은 특정 조건을 충족하는 주택의 임대차계약을 체결한 사람들을 대상으로 합니다.\n\n이 정책은 서울주거포털(https://housing.seoul.go.kr)에서 온라인으로 신청할 수 있습니다. 필요한 서류로는 주민등록등본, 가족관계증명서, 혼인관계증명서, 그리고 임대차계약서가 제출되어야 합니다.\n\n이 정책은 주거마련에 대한 부담을 완화하여 혼인수 감소와 출산기피 현상을 해결하고, 더 나은 주거환경을 제공하기 위해 서울시에서 운영하는 정책입니다. 대상 가구는 총 8,000가구로 제한되며, 지원 기간은 2023년 1월 1일부터 20

In [43]:
source_lst = []

for i in range(0, len(texts)):
    source_lst.append(texts[i].metadata['source'])

element_counts = Counter(source_lst)
filtered_counts = {key: value for key, value in element_counts.items() if value >= 2}
print('2개 이상으로 분할된 문서 :', filtered_counts)
print('분할된 텍스트 개수:', len(documents) + len(filtered_counts))

2개 이상으로 분할된 문서 : {'23.txt': 2, '22.txt': 2, '49.txt': 2, '31.txt': 2, '36.txt': 2, '48.txt': 2, '40.txt': 2}
분할된 텍스트 개수: 64


### ChromaDB를 이용한 검색기 사용하기

In [44]:
embedding = OpenAIEmbeddings()
vectordb = Chroma.from_documents(documents=texts, embedding=embedding)

In [45]:
retriever = vectordb.as_retriever()
docs = retriever.get_relevant_documents("신혼부부를 위한 정책이 있어?")
print("유사도가 높은 텍스트 개수 :", len(docs))
print('--' * 20)
print('유사도가 높은 텍스트 중 첫 번째 텍스트 출력 :', docs[0])
print('--' * 20)
print('유사도가 높은 텍스트들의 문서 출처 :')
for doc in docs:
    print(doc.metadata['source'])

유사도가 높은 텍스트 개수 : 4
----------------------------------------
유사도가 높은 텍스트 중 첫 번째 텍스트 출력 : page_content='이 정책은 주거마련에 대한 부담을 완화하여 혼인수 감소와 출산기피 현상을 해결하고, 더 나은 주거환경을 제공하기 위해 서울시에서 운영하는 정책입니다. 대상 가구는 총 8,000가구로 제한되며, 지원 기간은 2023년 1월 1일부터 2023년 12월 31일까지입니다.\n\n이 정책에 참여하기 위해서는 다음의 신청자격을 충족해야 합니다. 먼저, 서울시민이거나 대출 후 1개월 이내에 서울로 전입 예정이어야 합니다. 또한 혼인신고일 기준으로 7년 이내의 신혼부부이거나 서울시 추천서 신청일로부터 6개월 이내에 결혼식 예정인 예비신혼부부여야 합니다. 부부의 합산 연소득은 9천 7백만원 이하여야 하며, 본인 및 배우자는 무주택자여야 합니다. 또한, 특정 주택 조건을 충족하는 주택의 임대차계약을 체결한 자에게 대출이 지원됩니다.\n\n이 정책은 서울주거포털(https://housing.seoul.go.kr)을 통해 온라인으로 신청할 수 있습니다. 필요한 제출서류로는 주민등록등본, 가족관계증명서, 혼인관계증명서, 그리고 임대차계약서가 있습니다.\n\n이 정책은 서울시청 주택정책과에서 운영되며, 자세한 사항은 해당 사이트(https://housing.seoul.go.kr)에서 확인하실 수 있습니다.' metadata={'source': '23.txt'}
----------------------------------------
유사도가 높은 텍스트들의 문서 출처 :
23.txt
23.txt
39.txt
40.txt


In [46]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})
docs = retriever.get_relevant_documents("신혼부부를 위한 정책이 있어?")

for doc in docs:
    print(doc.metadata['source'])

23.txt
23.txt


### 질문으로부터 답변 얻기

In [47]:
qa_chain = RetrievalQA.from_chain_type(
    llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0),
    chain_type="stuff",
    retriever = retriever,
    return_source_documents = True
)

In [48]:
input_text = "대출과 관련된 정책이 궁급합니다"
chatbot_response = qa_chain(input_text)
print(chatbot_response)

{'query': '대출과 관련된 정책이 궁급합니다', 'result': '어떤 종류의 대출 정책에 대해 궁금하신가요? 서울시 학자금대출 신용회복 지원사업이나 역세권청년주택 주거비지원 정책에 대해 알려드릴 수 있습니다.', 'source_documents': [Document(page_content='정책내용: 서울시 학자금대출 신용회복 지원사업\n서울특별시 미래청년기획단이 주최하는 금융 정책으로, 학자금 대출로 인해 신용이 떨어져 어려움을 겪고 있는 청년층을 위한 신용회복 지원입니다. 이 정책은 분할상환약정 체결을 지원하고 초입금을 제공함으로써 신용유의자 등록을 해제하는 내용을 포함하고 있습니다. 추가적인 자부담 없이 약정을 체결할 수 있도록 합니다.\n\n지원 대상은 서울에 거주하며 학자금 대출로 인한 신용유의자인 19세부터 39세까지의 청년들이며, 약 200여명을 지원합니다. 2018년부터 2022년에 지원을 받은 사람은 2023년 지원 대상에서 제외됩니다.\n\n신청은 서울청년포털(youth.seoul.go.kr)을 통해 신청할 수 있으며, 심사 및 발표는 매월 1~2회 선정되며, 신청인원에 따라 주기가 변동할 수 있습니다.\n\n이 정책은 서울시 미래청년기획단이 운영하며, 자세한 내용은 관련 사이트를 참고하시기 바랍니다.\n\n[참고사이트]\n신용회복 신청 안내: https://youth.seoul.go.kr/site/main/board/notice/27789?baCategory1=basic&baCommSelec=true\n신청사이트: https://youth.seoul.go.kr/site/main/applyLoanCredit/step1', metadata={'source': '35.txt'}), Document(page_content='정책제목: 역세권청년주택 주거비지원\n서울시 전략주택공급과에서는 청년과 신혼부부의 주거비 부담을 경감하기 위해 임차보증금 대출을 지원하는 정책을 운영하고 있습니다. 이 정책은 혼인 감소 및 출산율 감소 문제를 해결하기

In [49]:
def get_chatbot_response(chatbot_response):
    print(chatbot_response['result'].strip())
    print('\n문서 출처:')
    for source in chatbot_response['source_documents']:
        print(source.metadata['source'])

In [50]:
input_text = "신혼부부의 신혼집 마련을 위한 정책이 있을까?"
chatbot_response = qa_chain(input_text)
get_chatbot_response(chatbot_response)

네, 신혼부부의 신혼집 마련을 위한 정책이 있습니다. 서울시에서는 "신혼부부 임차보증금 지원" 정책을 운영하고 있습니다. 이 정책은 주거마련에 대한 부담을 완화하여 혼인수 감소와 출산기피 현상을 해결하고, 더 나은 주거환경을 제공하기 위해 서울시에서 운영하는 정책입니다. 대상 가구는 총 8,000가구로 제한되며, 지원 기간은 2023년 1월 1일부터 2023년 12월 31일까지입니다. 자세한 내용은 서울주거포털(https://housing.seoul.go.kr)에서 확인하실 수 있습니다.

문서 출처:
23.txt
23.txt


In [51]:
print(chatbot_response)

{'query': '신혼부부의 신혼집 마련을 위한 정책이 있을까?', 'result': '네, 신혼부부의 신혼집 마련을 위한 정책이 있습니다. 서울시에서는 "신혼부부 임차보증금 지원" 정책을 운영하고 있습니다. 이 정책은 주거마련에 대한 부담을 완화하여 혼인수 감소와 출산기피 현상을 해결하고, 더 나은 주거환경을 제공하기 위해 서울시에서 운영하는 정책입니다. 대상 가구는 총 8,000가구로 제한되며, 지원 기간은 2023년 1월 1일부터 2023년 12월 31일까지입니다. 자세한 내용은 서울주거포털(https://housing.seoul.go.kr)에서 확인하실 수 있습니다.', 'source_documents': [Document(page_content='이 정책은 주거마련에 대한 부담을 완화하여 혼인수 감소와 출산기피 현상을 해결하고, 더 나은 주거환경을 제공하기 위해 서울시에서 운영하는 정책입니다. 대상 가구는 총 8,000가구로 제한되며, 지원 기간은 2023년 1월 1일부터 2023년 12월 31일까지입니다.\n\n이 정책에 참여하기 위해서는 다음의 신청자격을 충족해야 합니다. 먼저, 서울시민이거나 대출 후 1개월 이내에 서울로 전입 예정이어야 합니다. 또한 혼인신고일 기준으로 7년 이내의 신혼부부이거나 서울시 추천서 신청일로부터 6개월 이내에 결혼식 예정인 예비신혼부부여야 합니다. 부부의 합산 연소득은 9천 7백만원 이하여야 하며, 본인 및 배우자는 무주택자여야 합니다. 또한, 특정 주택 조건을 충족하는 주택의 임대차계약을 체결한 자에게 대출이 지원됩니다.\n\n이 정책은 서울주거포털(https://housing.seoul.go.kr)을 통해 온라인으로 신청할 수 있습니다. 필요한 제출서류로는 주민등록등본, 가족관계증명서, 혼인관계증명서, 그리고 임대차계약서가 있습니다.\n\n이 정책은 서울시청 주택정책과에서 운영되며, 자세한 사항은 해당 사이트(https://housing.seoul.go.kr)에서 확인하실 수 있습니다.', metadata={'s

### Gradio로 챗봇 UI 만들기

In [52]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 59.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.2/298.2 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 9.1 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.1-py3-none-any.whl size=5579 sha256=6b79d119bbb7eb138bac00cf4bede91f3033af703be99d4abddc00b71bfd3df5
  Stored in directory: /root/.cache/pip/wheels/01/a6/d1/1c0828c304a4283b2c1639a09ad86f83d7c487ef34c6b4a1bf
Successfully built ffmpy


In [56]:

import gradio as gr

# 인터페이스를 생성.
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="청년정책챗봇") # 청년정책챗봇 레이블을 좌측 상단에 구성
    msg = gr.Textbox(label="질문해주세요!")  # 하단의 채팅창의 레이블
    clear = gr.Button("대화 초기화")  # 대화 초기화 버튼

    # 챗봇의 답변을 처리하는 함수
    def respond(message, chat_history):
      result = qa_chain(message)
      bot_message = result['result']
      bot_message += ' # sources :'

      # 답변의 출처를 표기
      for i, doc in enumerate(result['source_documents']):
          bot_message += '[' + str(i+1) + '] ' + doc.metadata['source'] + ' '

      # 채팅 기록에 사용자의 메시지와 봇의 응답을 추가.
      chat_history.append((message, bot_message))
      return "", chat_history

    # 사용자의 입력을 제출(submit)하면 respond 함수가 호출.
    msg.submit(respond, [msg, chatbot], [msg, chatbot])

    # '초기화' 버튼을 클릭하면 채팅 기록을 초기화.
    clear.click(lambda: None, None, chatbot, queue=False)

# 인터페이스 실행.
demo.launch(debug=True, share=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://a6a1db888025fa302c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://a6a1db888025fa302c.gradio.live
